In [2]:
import pandas as pd
import numpy as np

In [5]:
info_df = pd.read_csv('snopes_ground_truth.csv')
info_df.head()

,snopes_page,topic,claim,claim_label,date_published,date_updated,page_url,page_is_example,page_is_image_credit,page_is_archived,page_is_first_citation,tags
0,http://www.snopes.com/rowan-atkinson-death-hoax/,fake news,"British actor Rowan Atkinson, best known as 'M...",False,"Jul 15th, 2016","Mar 18th, 2017",https://www.shutterstock.com/image-photo/mosco...,False,True,False,False,NaN
1,http://www.snopes.com/european-parental-leave-...,politics,"Canada, Norway, and Germany each offer federal...",True,"Mar 16th, 2017",NaN,https://www.facebook.com/senatorsanders/photos...,False,False,False,True,"canada,sen-kirsten-gillibrand,sen-bernie-sande..."
2,http://www.snopes.com/are-blue-balls-a-real-th...,love,Prolonged sexual arousal in men without an eja...,True,"Mar 15th, 2017",NaN,http://pediatrics.aappublications.org/content/...,False,False,False,True,NaN
3,http://www.snopes.com/politics/soapbox/knox.asp,political news,The president of Knox Machinery wrote a letter...,True,"Dec 16th, 2008","Mar 14th, 2017",http://knoxmachinery.com,False,False,False,True,NaN
4,http://www.snopes.com/photographs-of-eiffel-to...,NaN,It is illegal to take photographs of the Eiffe...,True,"Mar 13th, 2017",NaN,http://www.toureiffel.paris/,False,False,False,True,"paris,photography,eiffel-tower,copyright"


In [33]:
topic_fake_count = info_df.groupby(['topic', 'claim_label'])['claim_label'].count()
topic_fake_count

topic                claim_label
astronomy news       True            1
business             False           5
                     True           10
cokelore             True            1
college              False           1
                     True            1
computers            False           2
                     True            1
crime                False           4
                     True            8
critter country      False           2
                     True            9
disney               False           2
                     True            1
entertainment        False           7
                     True            7
fake news            False          81
                     True            1
fauxtography         False          51
                     True           44
food                 False           2
                     True            4
glurge gallery       True            3
history              False           4
                     True      

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [73]:
vectorizer = TfidfVectorizer(input='filename',max_df=0.5, max_features=50,
                                 min_df=2, stop_words='english',encoding='utf-8',decode_error ='ignore',
                                 use_idf=True)

In [89]:
import os
a = os.listdir('webscrapedtext/')
b =[]
for i in a:
    i = 'webscrapedtext/'+i
    b.append(str(i)) 
vect_matrix = vectorizer.fit_transform(b)

In [90]:
vect_matrix.shape

(562, 50)

In [104]:
train_features, test_features = vect_matrix[:400,],vect_matrix[401:563,]
train_labels, test_labels = info_df['claim_label'][:400] , info_df['claim_label'][401:563]

In [105]:
from sklearn import linear_model
logistic = linear_model.LogisticRegression()

logr= logistic.fit(train_features,train_labels)

In [108]:
logr.predict(test_features)

array([False, False, False,  True,  True,  True, False, False, False,
       False,  True, False, False,  True,  True,  True,  True, False,
        True,  True, False,  True,  True,  True, False,  True,  True,
       False, False,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False,  True, False, False,  True,
        True,  True,  True,  True, False, False,  True,  True,  True,
        True, False,  True, False,  True,  True, False,  True, False,
       False,  True,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True,  True, False,  True, False,
       False,  True,  True,  True,  True, False,  True,  True, False,
        True,  True,  True, False,  True,  True, False, False, False,
       False, False, False, False, False,  True, False, False, False,
        True, False,

In [110]:
logr.score(test_features,test_labels)

0.453416149068323

In [112]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(train_features,train_labels)
mnb.score(test_features,test_labels)

0.391304347826087

In [113]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=3, random_state=0)
clf.fit(train_features,train_labels)
clf.score(test_features,test_labels)

0.4968944099378882